In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import os
import cv2
import numpy as np
from skimage.transform import rotate, resize
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [14]:
def preprocess_roi(roi):
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    eq = cv2.equalizeHist(gray)
    bin_img = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                     cv2.THRESH_BINARY, 23, 5)
    return eq, bin_img

In [15]:
def compute_fitness(template_img, eq_img, bin_img, tx, ty, sx, sy, theta):
    h, w = template_img.shape[:2]
    th, tw = int(h * sy), int(w * sx)

    temp = resize(template_img, (th, tw), anti_aliasing=True)
    temp = rotate(temp, theta, resize=False)
    temp = (temp * 255).astype(np.uint8)

    roi_h, roi_w = eq_img.shape
    if tx + tw > roi_w or ty + th > roi_h:
        return 0

    eq_crop = eq_img[ty:ty+th, tx:tx+tw]
    bin_crop = bin_img[ty:ty+th, tx:tx+tw]

    third = th // 3
    if third == 0: return 0

    a = np.mean(eq_crop[0:third])
    b = np.mean(eq_crop[third:2*third])
    c = np.mean(eq_crop[2*third:])
    O1 = (abs(a - b) + abs(c - b)) / 2

    awht = np.sum(bin_crop[0:third] == 255) / (bin_crop[0:third].size + 1e-5)
    bblk = np.sum(bin_crop[third:2*third] == 0) / (bin_crop[third:2*third].size + 1e-5)
    cwht = np.sum(bin_crop[2*third:] == 255) / (bin_crop[2*third:].size + 1e-5)
    O2 = (awht + cwht + 2 * bblk) / 4

    return O1 + O2

In [22]:
def run_ga(template, eq_img, bin_img, img_size, pop_size=50, gen_size=50):
    wtgt, htgt = img_size

    if wtgt <= 46 or htgt <= 18:
        print("⚠️ Skipping image: too small for GA template matching.")
        return None, 0

    population = [(
        np.random.randint(0, wtgt - 46),
        np.random.randint(0, htgt - 18),
        np.random.uniform(0.8, 1.2),
        np.random.uniform(0.8, 1.2),
        np.random.uniform(-30, 0)
    ) for _ in range(pop_size)]

    best_fit = -1
    best_ind = None

    for _ in range(gen_size):
        fitnesses = []
        for ind in population:
            tx, ty, sx, sy, theta = ind
            f = compute_fitness(template, eq_img, bin_img, tx, ty, sx, sy, theta)
            fitnesses.append(f)
            if f > best_fit:
                best_fit = f
                best_ind = ind

        # Top 20% selection
        parents = [population[i] for i in np.argsort(fitnesses)[-pop_size//5:]]
        new_population = []

        while len(new_population) < pop_size:
            p1, p2 = random.sample(parents, 2)

            child = (
                np.random.choice([p1[0], p2[0]]),
                np.random.choice([p1[1], p2[1]]),
                (p1[2] + p2[2]) / 2,
                (p1[3] + p2[3]) / 2,
                (p1[4] + p2[4]) / 2
            )
            if np.random.rand() < 0.1:
                child = (
                    child[0] + np.random.randint(-5, 5),
                    child[1] + np.random.randint(-5, 5),
                    child[2] * np.random.uniform(0.95, 1.05),
                    child[3] * np.random.uniform(0.95, 1.05),
                    child[4] + np.random.uniform(-2, 2)
                )
            new_population.append(child)

        population = new_population

    return best_ind, best_fit

In [20]:
def overlay_and_save(test_img, template, params, out_path):
    tx, ty, sx, sy, theta = params
    th, tw = template.shape[:2]
    h, w = int(th * sy), int(tw * sx)

    resized = resize(template, (h, w), anti_aliasing=True)
    rotated = rotate(resized, theta, resize=False)
    rotated = (rotated * 255).astype(np.uint8)
    if len(rotated.shape) == 2:
        rotated = cv2.cvtColor(rotated, cv2.COLOR_GRAY2BGR)

    overlay = test_img.copy()
    if ty + h > overlay.shape[0] or tx + w > overlay.shape[1]:
        print("⚠️ Out of bounds, skipping overlay.")
        return

    cv2.rectangle(overlay, (tx, ty), (tx + w, ty + h), (0, 255, 0), 2)
    roi = overlay[ty:ty+h, tx:tx+w]
    overlay[ty:ty+h, tx:tx+w] = cv2.addWeighted(roi, 0.5, rotated, 0.5, 0)

    cv2.imwrite(out_path, overlay)

In [23]:
template_path = "template.jpg"
input_folder = "/content/cropped_rois/test"
output_folder = "ga_results"
ensure_dir(output_folder)

template = cv2.imread(template_path)

for img_file in tqdm(os.listdir(input_folder)):
    if not img_file.lower().endswith((".jpg", ".png")):
        continue

    img_path = os.path.join(input_folder, img_file)
    out_path = os.path.join(output_folder, img_file)

    img = cv2.imread(img_path)
    eq, bin_img = preprocess_roi(img)

    best_ind, score = run_ga(template, eq, bin_img, img_size=eq.shape[::-1])
    if best_ind is None:
      continue
    print(f"{img_file}: Fitness={score:.2f}, Params={best_ind}")

    overlay_and_save(img, template, best_ind, out_path)

  0%|          | 1/1085 [00:02<45:54,  2.54s/it]

05-09-04_jpg.rf.3697cd52aa696b844db6277265a9bf25_person-noseatbelt_2.jpg: Fitness=47.96, Params=(np.int64(1), np.int64(63), 0.9066066925758605, 1.0192810660774099, -15.627964165925496)


  0%|          | 2/1085 [00:05<45:07,  2.50s/it]

1625227981033_jpg.rf.a9131630836bf994d5a538f55a584a0d_person-seatbelt_2.jpg: Fitness=64.13, Params=(np.int64(4), np.int64(30), 0.8787906620766177, 1.0650757120111145, -8.05639661645759)
⚠️ Skipping image: too small for GA template matching.


  0%|          | 4/1085 [00:07<31:13,  1.73s/it]

...

1_CAR_17-14-45-482_jpg.rf.ebd4bb794b5ba549a6c98246ab884a99_windshield_0.jpg: Fitness=80.63, Params=(np.int64(200), np.int64(154), 0.9218877237878219, 1.049839501043652, -23.497069904030017)


100%|█████████▉| 1084/1085 [48:08<00:02,  2.96s/it]

1625227981220_jpg.rf.975fa8a42995627a6c7203358b13ebcd_person-seatbelt_1.jpg: Fitness=50.60, Params=(np.int64(4), np.int64(3), 0.947551376604121, 0.9949149253453586, -15.560663274436145)


100%|██████████| 1085/1085 [48:11<00:00,  2.66s/it]

749_png_jpg.rf.f505d60f1f41f7edf70279932760a872_windshield_2.jpg: Fitness=102.14, Params=(np.int64(23), np.int64(104), 1.0134142958911587, 0.965069647041456, -14.592262861861494)


In [25]:
import os
import numpy as np
import pickle
from collections import defaultdict
from glob import glob

In [26]:
def build_ili(descriptor_folder):
    ili = defaultdict(list)
    for file in glob(os.path.join(descriptor_folder, "*.npy")):
        desc = np.load(file)
        key = tuple(desc)  # Binarized HOG
        ili[key].append(file)
    with open("ili_index.pkl", "wb") as f:
        pickle.dump(ili, f)
    print(f"ILI index built with {len(ili)} unique keys.")

In [27]:
def pqhog_search(test_image_path, template_folder, ili_path, top_k=5):
    from sklearn.preprocessing import Binarizer
    from skimage.feature import hog
    import cv2

    with open(ili_path, "rb") as f:
        ili_index = pickle.load(f)

    image = cv2.imread(test_image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Extract PQ-HOG
    features, _ = hog(gray, pixels_per_cell=(8,8), cells_per_block=(1,1),
                      orientations=9, visualize=True)
    binarized = Binarizer(threshold=0).fit_transform(features.reshape(1, -1)).flatten()
    query_key = tuple(binarized)

    # Simple matching: Hamming distance between binary vectors
    def hamming_dist(a, b):
        return np.sum(np.array(a) != np.array(b))

    distances = []
    for key in ili_index:
        d = hamming_dist(query_key, key)
        distances.append((d, key))

    distances.sort()
    top_keys = [key for _, key in distances[:top_k]]

    matches = []
    for key in top_keys:
        for path in ili_index[key]:
            matches.append(path)
    return matches[:top_k]


In [28]:
def hybrid_detect(test_image_path, template_folder, descriptor_folder, ili_path, output_path):
    matches = pqhog_search(test_image_path, template_folder, ili_path)

    test_img = cv2.imread(test_image_path)
    eq, bin_img = preprocess_roi(test_img)
    best_fit = -1
    best_result = None
    best_template = None

    for match_path in matches:
        template = cv2.imread(match_path)
        result, score = run_ga(template, eq, bin_img, img_size=eq.shape[::-1])
        if result and score > best_fit:
            best_fit = score
            best_result = result
            best_template = template

    if best_result:
        filename = os.path.basename(test_image_path)
        overlay_and_save(test_img, best_template, best_result, os.path.join(output_path, filename))
        print(f"✅ Saved best hybrid match for {filename} with fitness {best_fit:.2f}")
    else:
        print("⚠️ No valid match found.")


In [ ]:
# Step 1: Build the ILI index (run once)
build_ili("pqhog_descriptors/test")

# Step 2: Run on one image (test hybrid matching)
hybrid_detect(
    test_image_path="test_images/example.jpg",
    template_folder="templates",
    descriptor_folder="pqhog_descriptors/test",
    ili_path="ili_index.pkl",
    output_path="results"
)
